# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

/Users/clawall/estudo/coursera/Applied-Plotting-Charting-Data-Representation-in-Python-Assignment-2/.env/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [18]:
binsize = 400
hashid = 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89'

df_stations = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

df_stations = df_stations[df_stations['hash'] == hashid]

In [28]:
my_data_filename = 'data/C2A2_data/BinnedCsvs_d{}/{}.csv'.format(binsize, hashid)
df_my_data = pd.read_csv(my_data_filename, parse_dates=[0])

full_df = pd.merge(df_stations[['ID']], df_my_data, how='left', on='ID', sort=False)
full_df['Data_Value'] = full_df['Data_Value'].apply(lambda x: x/10)

In [29]:
del full_df['ID']

In [30]:
full_df.drop_duplicates(inplace=True)

In [31]:
def prepare_df(df, agg_func_name):
    res = df[df['Data_Value'] == df.groupby('Date')['Data_Value'].transform(agg_func_name)]
    del res['Element']
    res.columns = ['date', agg_func_name]
    return res

In [40]:
# creating min temperature's df
min_df = prepare_df(full_df, 'min')

# creating max temperature's df
max_df = prepare_df(full_df, 'max')

# merging into single df
range_df = pd.merge(min_df, max_df, how='left', sort=False)

# converting 'date' column to datetime
range_df['date'] = pd.to_datetime(range_df['date'], format='%Y-%m-%d')

# removing leap days
range_df = range_df.drop(range_df[((range_df.date.dt.month == 2) & (range_df.date.dt.day == 29))].index)

In [47]:
df_2005_20014 = range_df[(range_df['date'] >= '2005-01-01') & (range_df['date'] <= '2014-12-31')]
df_2015 = range_df[(range_df['date'] >= '2015-01-01') & (range_df['date'] <= '2015-12-31')]

In [48]:
df_2015.sort_values('date')

,date,min,max
2589,2015-01-01,-13.3,1.1
2601,2015-01-02,-12.2,3.9
534,2015-01-03,-6.7,3.9
3139,2015-01-04,-8.8,4.4
3141,2015-01-05,-15.5,2.8
...,...,...,...
163,2015-12-27,-0.6,8.3
1556,2015-12-28,-3.9,6.1
3084,2015-12-29,-3.9,10.0
1715,2015-12-30,-2.2,6.7
